In [1]:
# Dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.orm import Session

from sqlalchemy.ext.declarative import declarative_base

import pymysql
pymysql.install_as_MySQLdb()

In [2]:
Base = declarative_base()

class Stations(Base):
    __tablename__ = 'stations'
    #__table_args__ = {"extend_existing": True}
    
    idx = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)   

class Measurements(Base):
    __tablename__ = 'measurements'
    #__table_args__ = {"extend_existing": True}
    
    idx = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)

In [3]:
# Create Database connection
engine = create_engine("sqlite:///./Resources/hawaii.sqlite", echo=True)
conn = engine.connect()

# create metadata layer
Base.metadata.create_all(engine)

# create session object to connect to db
session = Session(bind=engine)

2018-07-15 19:02:13,903 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-15 19:02:13,905 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:02:13,907 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-15 19:02:13,908 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:02:13,915 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stations")
2018-07-15 19:02:13,916 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:02:13,919 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurements")
2018-07-15 19:02:13,920 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:02:13,922 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE stations (
	idx INTEGER NOT NULL, 
	station VARCHAR, 
	name VARCHAR, 
	latitude FLOAT, 
	longitude FLOAT, 
	elevation FLOAT, 
	PRIMARY KEY (idx)
)


2018-07-15 19:02:13,923 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:02:13,926 INFO sqlalchemy.engine.b

In [4]:
# Read in data
measurements_df = pd.read_csv("Resources/clean_hawaii_measurements.csv")
measurements_df['idx'] = measurements_df.index
measurements_df = measurements_df[['idx', 'station', 'date', 'prcp', 'tobs']]
print(measurements_df.head(2))

stations_df = pd.read_csv("Resources/clean_hawaii_stations.csv")
stations_df['idx'] = stations_df.index
stations_df = stations_df[['idx', 'station', 'name', 'latitude', 'longitude', 'elevation']]
print(stations_df.head(2))

   idx      station        date  prcp  tobs
0    0  USC00519397  2010-01-01  0.08    65
1    1  USC00519397  2010-01-02  0.00    63
   idx      station                  name  latitude  longitude  elevation
0    0  USC00519397  WAIKIKI 717.2, HI US   21.2716  -157.8168        3.0
1    1  USC00513117  KANEOHE 838.1, HI US   21.4234  -157.8015       14.6


In [5]:
# Set up dtype maps
dtype_map_measurements = {
    'idx': Integer,
    'station': String,
    'date': String, 
    'prcp': Float, 
    'tobs': Integer
}
dtype_map_stations = {
    'idx': Integer,
    'station': String,
    'name': String, 
    'latitude': Float, 
    'longitude': Float, 
    'elevation': Float
}
# Create station and measurement tables within the database
measurements_df.to_sql("measurements", engine, dtype=dtype_map_measurements, if_exists='replace', index=False)
stations_df.to_sql("stations", engine, dtype=dtype_map_stations, if_exists='replace', index=False)

2018-07-15 19:03:22,824 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurements")
2018-07-15 19:03:22,825 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:03:22,828 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurements")
2018-07-15 19:03:22,829 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:03:22,833 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-15 19:03:22,833 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:03:22,839 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurements")
2018-07-15 19:03:22,840 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:03:22,842 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurements' AND type = 'table'
2018-07-15 19:03:22,843 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 19:03:22,845 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("measur

In [6]:
inspector = inspect(engine)
inspector.get_table_names()

2018-07-15 19:04:32,083 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-15 19:04:32,084 INFO sqlalchemy.engine.base.Engine ()


['measurements', 'stations']

In [7]:
# Get a list of column names and types
columns = inspector.get_columns('stations')
for c in columns:
    print(c['name'], c["type"])

print()
# Get a list of column names and types
columns = inspector.get_columns('measurements')
for c in columns:
    print(c['name'], c["type"])

2018-07-15 19:04:53,607 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stations")
2018-07-15 19:04:53,608 INFO sqlalchemy.engine.base.Engine ()
idx INTEGER
station VARCHAR
name VARCHAR
latitude FLOAT
longitude FLOAT
elevation FLOAT

2018-07-15 19:04:53,612 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurements")
2018-07-15 19:04:53,612 INFO sqlalchemy.engine.base.Engine ()
idx INTEGER
station VARCHAR
date VARCHAR
prcp FLOAT
tobs INTEGER


In [62]:
# station_objs = []
# for s in stations_df.itertuples():
#     data = Stations(
#         idx = s.idx,
#         station = s.station,
#         name = s.name, 
#         latitude = s.latitude,
#         longitude = s.longitude,
#         elevation = s.elevation
#     )
#     station_objs.append(data)
# print(station_objs)
# session.add_all(station_objs)
# session.commit()

[<__main__.Stations object at 0x10ebce4e0>, <__main__.Stations object at 0x10ebce2b0>, <__main__.Stations object at 0x10fec72b0>, <__main__.Stations object at 0x110d08b38>, <__main__.Stations object at 0x10fda5630>, <__main__.Stations object at 0x10eb0d128>, <__main__.Stations object at 0x10eb0d048>, <__main__.Stations object at 0x10eb0d2e8>, <__main__.Stations object at 0x10eb0d438>]
2018-07-15 18:59:33,021 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 18:59:33,024 INFO sqlalchemy.engine.base.Engine INSERT INTO stations (idx, station, name, latitude, longitude, elevation) VALUES (?, ?, ?, ?, ?, ?)
2018-07-15 18:59:33,025 INFO sqlalchemy.engine.base.Engine ((0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0), (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6), (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0), (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9), (4, 'USC00518838', 'UPPE

OperationalError: (sqlite3.OperationalError) table stations has no column named idx [SQL: 'INSERT INTO stations (idx, station, name, latitude, longitude, elevation) VALUES (?, ?, ?, ?, ?, ?)'] [parameters: ((0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0), (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6), (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0), (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9), (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6), (5, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5), (6, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9), (7, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9), (8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4))] (Background on this error at: http://sqlalche.me/e/e3q8)

In [38]:
# metadata = MetaData(bind=engine)

# measurements = Table('measurements', metadata,
#                     Column('id', Integer, primary_key=True),
#                     Column('station', String),
#                     Column('date', String),
#                     Column('prcp', Float),
#                     Column('tobs', Integer)
#                     )
# stations = Table('stations', metadata,
#                 Column('id', Integer, primary_key=True),
#                 Column('station', String),
#                 Column('name', String),
#                 Column('latitude', Float),
#                 Column('longitude', Float),
#                 Column('elevation', Float)
#                 )

# metadata.create_all(engine)

# conn.execute(measurements.insert(), measurements_df.to_dict('records'))
# conn.execute(stations.insert(), stations_df.to_dict('records'))

2018-07-15 18:33:25,343 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurements")
2018-07-15 18:33:25,344 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 18:33:25,346 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stations")
2018-07-15 18:33:25,347 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 18:33:25,912 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-07-15 18:33:25,913 INFO sqlalchemy.engine.base.Engine (('USC00519397', '2010-01-01', 0.08, 65), ('USC00519397', '2010-01-02', 0.0, 63), ('USC00519397', '2010-01-03', 0.0, 74), ('USC00519397', '2010-01-04', 0.0, 76), ('USC00519397', '2010-01-07', 0.06, 70), ('USC00519397', '2010-01-08', 0.0, 64), ('USC00519397', '2010-01-09', 0.0, 68), ('USC00519397', '2010-01-10', 0.0, 73)  ... displaying 10 of 18103 total bound parameter sets ...  ('USC00516128', '2017-08-22', 0.5, 76), ('USC00516128', '2017-08-23', 0.45, 76))
2018-07-15 18:33:25,971 INFO sqlalchemy.

------------------

In [53]:
# Create a session
session = Session(bind=engine)

In [8]:
m = session.query(Measurements).first()
m.__dict__

2018-07-15 19:05:24,516 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 19:05:24,520 INFO sqlalchemy.engine.base.Engine SELECT measurements.idx AS measurements_idx, measurements.station AS measurements_station, measurements.date AS measurements_date, measurements.prcp AS measurements_prcp, measurements.tobs AS measurements_tobs 
FROM measurements
 LIMIT ? OFFSET ?
2018-07-15 19:05:24,521 INFO sqlalchemy.engine.base.Engine (1, 0)


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x116e9ba58>,
 'date': datetime.date(2010, 1, 1),
 'idx': 0,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [9]:
Base.classes.keys()

AttributeError: type object 'Base' has no attribute 'classes'

In [10]:
s = session.query(Stations).first()
s.__dict__

2018-07-15 19:05:49,851 INFO sqlalchemy.engine.base.Engine SELECT stations.idx AS stations_idx, stations.station AS stations_station, stations.name AS stations_name, stations.latitude AS stations_latitude, stations.longitude AS stations_longitude, stations.elevation AS stations_elevation 
FROM stations
 LIMIT ? OFFSET ?
2018-07-15 19:05:49,852 INFO sqlalchemy.engine.base.Engine (1, 0)


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x116136a20>,
 'elevation': 3.0,
 'idx': 0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [12]:
precip = session.query(Measurements).filter(Measurements.date == '2010-01-09').all()
for p in precip:
#     print(p)
    print(p.date, p.prcp, p.idx)

2018-07-15 19:06:14,399 INFO sqlalchemy.engine.base.Engine SELECT measurements.idx AS measurements_idx, measurements.station AS measurements_station, measurements.date AS measurements_date, measurements.prcp AS measurements_prcp, measurements.tobs AS measurements_tobs 
FROM measurements 
WHERE measurements.date = ?
2018-07-15 19:06:14,401 INFO sqlalchemy.engine.base.Engine ('2010-01-09',)
2010-01-09 0.0 6
2010-01-09 0.0 2693
2010-01-09 0.01 8007
2010-01-09 0.01 8351
2010-01-09 0.0 10923
2010-01-09 0.0 13695
2010-01-09 0.0 15627


In [13]:
s = session.query('stations')
s.__dict__

{'_entities': [<sqlalchemy.orm.query._ColumnEntity at 0x11614f080>],
 '_has_mapper_entities': False,
 '_mapper_adapter_map': {},
 '_polymorphic_adapters': {},
 '_primary_entity': None,
 'session': <sqlalchemy.orm.session.Session at 0x115f9cf98>}

In [14]:
d = session.query(Stations)
for item in d:
    print(item.station, item.idx)

2018-07-15 19:06:25,262 INFO sqlalchemy.engine.base.Engine SELECT stations.idx AS stations_idx, stations.station AS stations_station, stations.name AS stations_name, stations.latitude AS stations_latitude, stations.longitude AS stations_longitude, stations.elevation AS stations_elevation 
FROM stations
2018-07-15 19:06:25,263 INFO sqlalchemy.engine.base.Engine ()
USC00519397 0
USC00513117 1
USC00514830 2
USC00517948 3
USC00518838 4
USC00519523 5
USC00519281 6
USC00511918 7
USC00516128 8
